In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle

torch.manual_seed(42)

# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca




# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]

    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=batch_size,shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=batch_size, shuffle=False)



In [13]:
grid=1
device = 'cpu'

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
import math
from functions.training import Early_stop_train_KAN
from kan import KAN

# 데이터 로드 및 전처리
"""
data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=PCA_dim)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.PCA_dim, random_state=seed)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)"""

# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)
            
        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch')
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch')
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))


class Feature_model(nn.Module):
    def __init__(self,grid,device='cpu'):
        super(Feature_model,self).__init__()
        KAN_model = KAN([PCA_dim,2*PCA_dim+1,PCA_dim*2-1],grid=grid,device=device)
        self.KAN = KAN_model
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        input2 = inputs[1]

        input1 = nn.Sigmoid()(self.KAN(input1))*np.pi
        input2 = nn.Sigmoid()(self.KAN(input2))*np.pi

        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        
        return output[:,0].clamp(min=epsilon, max=1-epsilon)
        
    



# 하이브리드 모델 정의

grid = 2
feature_model = Feature_model(grid,device); criterion = nn.BCELoss()


optimizer = optim.Adam(feature_model.parameters(), lr=0.01)



# 모델 학습 및 평가
train_process = Early_stop_train_KAN(feature_model, optimizer, criterion)
train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15,lamb=0)
feature_model.KAN = feature_model.KAN.prune()
res,pretrain_acc = train_process.test(test_feature_loader)
print(f"\n Pretrain acc : {pretrain_acc}")


# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.KAN = feature_model.KAN

        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        epsilon = 1e-6
        x = self.KAN(x)*np.pi
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return quantum_output[:,0].clamp(min=epsilon, max=1-epsilon)
    
model = HybridModel(); criterion = nn.BCELoss()
optimizer = optim.Adam([model.Q_params], lr=0.01)
print("\n\nTest start\n\n")
train_process = Early_stop_train_KAN(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=25,lamb=0)

_,acc = train_process.test(test_loader)





Test set: Average loss: 4.8628, Accuracy: 156.00000071525574/300 (52%)
Epoch 1 Loss 0.765622 acc : 0.600568 reg : 0.000000 stop count : 0
Test set: Average loss: 3.4188, Accuracy: 188.9999988079071/300 (63%)
Epoch 2 Loss 0.578161 acc : 0.705587 reg : 0.000000 stop count : 0
Test set: Average loss: 3.2568, Accuracy: 201.00000023841858/300 (67%)
Epoch 3 Loss 0.522701 acc : 0.731723 reg : 0.000000 stop count : 0
Test set: Average loss: 3.0613, Accuracy: 207.0000011920929/300 (69%)
Epoch 4 Loss 0.480870 acc : 0.761837 reg : 0.000000 stop count : 0
Test set: Average loss: 2.9001, Accuracy: 204.00000023841858/300 (68%)
Epoch 5 Loss 0.450186 acc : 0.794508 reg : 0.000000 stop count : 0
Test set: Average loss: 2.7848, Accuracy: 212.00000095367432/300 (71%)
Epoch 6 Loss 0.420805 acc : 0.804072 reg : 0.000000 stop count : 0
Test set: Average loss: 2.6658, Accuracy: 213.99999904632568/300 (71%)
Epoch 7 Loss 0.381212 acc : 0.841572 reg : 0.000000 stop count : 0
Test set: Average loss: 2.5774, Acc

AttributeError: 'HybridModel' object has no attribute 'KAN'

In [15]:
## 커널 메소드
from functions.training import Kernal_method

kernal = Kernal_method(feature_model)

kernel_matrix,test_kernel_matrix,alpha,labels = kernal.train(x_train_pca,y_train,x_test_pca,y_test)

100%|██████████| 700/700 [07:41<00:00,  1.52it/s]
c:\Users\pop75\OneDrive\Desktop\Quantum_KAN\functions\training.py:215: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_train).float()
c:\Users\pop75\OneDrive\Desktop\Quantum_KAN\functions\training.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test).float()
100%|██████████| 700/700 [04:57<00:00,  2.35it/s]

 acc :  0.8899999856948853


(tensor([[1.0000, 0.9539, 0.0398,  ..., 0.9406, 0.9788, 0.0257],
         [0.9539, 1.0000, 0.1100,  ..., 0.8054, 0.8819, 0.0811],
         [0.0398, 0.1100, 1.0000,  ..., 0.0071, 0.0172, 0.9944],
         ...,
         [0.9406, 0.8054, 0.0071,  ..., 1.0000, 0.9856, 0.0035],
         [0.9788, 0.8819, 0.0172,  ..., 0.9856, 1.0000, 0.0099],
         [0.0257, 0.0811, 0.9944,  ..., 0.0035, 0.0099, 1.0000]]),
 tensor([[0.0714, 0.1680, 0.9721,  ..., 0.0173, 0.0352, 0.9502],
         [0.9432, 0.9952, 0.1210,  ..., 0.7925, 0.8705, 0.0911],
         [0.9423, 0.8126, 0.0079,  ..., 0.9956, 0.9899, 0.0040],
         ...,
         [0.9476, 0.8191, 0.0084,  ..., 0.9983, 0.9915, 0.0043],
         [0.4532, 0.6489, 0.5807,  ..., 0.2522, 0.3370, 0.5130],
         [0.8459, 0.9556, 0.2047,  ..., 0.6598, 0.7494, 0.1646]]),
 tensor([0.5157, 0.5150, 0.5215, 0.5152, 0.5151, 0.5153, 0.5210, 0.5155, 0.5155,
         0.5210, 0.5153, 0.5156, 0.5215, 0.5216, 0.5153, 0.5157, 0.5155, 0.5196,
         0.5104, 0.5154, 0

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
